In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai4eng-20242
!unzip udea*.zip > /dev/null


import pandas as pd
import numpy as np

z = pd.read_csv("train.csv")
w = pd.read_csv("test.csv")
print ("(rows, columns))", z.shape)


udea-ai4eng-20242.zip: Skipping, found more recently modified local copy (use --force to force download)
replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
(rows, columns)) (692500, 12)


In [ ]:
z0 = z[['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'RENDIMIENTO_GLOBAL']]
test_data = w[['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET']]
z0.head()

,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,RENDIMIENTO_GLOBAL
0,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,medio-alto
1,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,bajo
2,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,bajo
3,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,alto
4,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,medio-bajo


In [ ]:
def transformar_internet(df):
    # Asegurarse de que la columna sea de tipo cadena antes de llenar valores nulos
    df.loc[:, 'FAMI_TIENEINTERNET'] = df['FAMI_TIENEINTERNET'].fillna('no info')
    internet_map={
        'no info': 0,
        'No': 0,
        'Si': 1
    }
    df.loc[:, 'FAMI_TIENEINTERNET'] = df['FAMI_TIENEINTERNET'].map(internet_map).astype(str)
    return df

def transformar_estrato(df):
    # Asegurarse de que la columna sea de tipo cadena antes de manipular los valores
    df.loc[:, 'FAMI_ESTRATOVIVIENDA'] = df['FAMI_ESTRATOVIVIENDA'].astype(str)
    df.loc[:, 'FAMI_ESTRATOVIVIENDA'] = df['FAMI_ESTRATOVIVIENDA'].str.extract('(\d+)')
    df.loc[:, 'FAMI_ESTRATOVIVIENDA'] = df['FAMI_ESTRATOVIVIENDA'].fillna('0')
    return df

def transformar_matricula(df):
    # Asegurarse de que la columna sea de tipo cadena antes de llenarla con 'no info'
    cmap = {'Entre 1 millón y menos de 2.5 millones': 1.75,
        'Entre 2.5 millones y menos de 4 millones': 3.25,
        'Menos de 500 mil': .250,
        'Entre 500 mil y menos de 1 millón': .75,
        'Entre 4 millones y menos de 5.5 millones': 4.75,
        'Más de 7 millones': 7.75,
        'Entre 5.5 millones y menos de 7 millones': 6.25,
        'No pagó matrícula': 0,
        'no info': -1}
    df.loc[:, 'ESTU_VALORMATRICULAUNIVERSIDAD'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna('no info')
    df.loc[:, 'ESTU_VALORMATRICULAUNIVERSIDAD'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].map(cmap).astype(str)
    return df

def transformar_horas_trabajo(df):
    # Asegurarse de que la columna sea de tipo cadena antes de rellenarla con '0'
    horas_map = {
        'Más de 30 horas':1,
        '0':0,
        'Menos de 10 horas':1,
        'Entre 11 y 20 horas':1,
        'Entre 21 y 30 horas':1
    }
    df.loc[:, 'ESTU_HORASSEMANATRABAJA'] = df['ESTU_HORASSEMANATRABAJA'].fillna('0')
    df.loc[:, 'ESTU_HORASSEMANATRABAJA'] = df['ESTU_HORASSEMANATRABAJA'].map(horas_map).astype(str)
    return df

def transformar_rendimiento(df):
    # Convertir los valores de RENDIMIENTO_GLOBAL en números
    y_col = 'RENDIMIENTO_GLOBAL'
    rmap = {'alto': 3, 'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2}

    # Aplicar el mapeo y reemplazar NaN por un valor por defecto (ej. 0)
    df.loc[:, y_col] = df[y_col].map(rmap).fillna(0).astype(int)

    return df

def transformar_datos(train_data, test_data):
    # Aplicar las transformaciones a los conjuntos de datos
    train_data = transformar_internet(train_data)
    train_data = transformar_estrato(train_data)
    train_data = transformar_matricula(train_data)
    train_data = transformar_horas_trabajo(train_data)
    train_data = transformar_rendimiento(train_data)

    test_data = transformar_internet(test_data)
    test_data = transformar_estrato(test_data)
    test_data = transformar_matricula(test_data)
    test_data = transformar_horas_trabajo(test_data)


    return train_data, test_data


# Transformar ambos conjuntos
train_data_transformed, test_data_transformed = transformar_datos(z0, test_data)

In [ ]:
train_data_transformed

,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,RENDIMIENTO_GLOBAL
0,6.25,1,3,1,2
1,3.25,0,3,0,0
2,3.25,1,3,1,0
3,4.75,0,4,1,3
4,3.25,1,3,1,1
...,...,...,...,...,...
692495,0.75,1,2,1,2
692496,3.25,1,3,1,0
692497,1.75,1,3,1,1
692498,3.25,1,1,0,0


In [ ]:
test_data_transformed.head()

,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET
0,0.25,1,3,1
1,3.25,1,2,1
2,1.75,0,3,1
3,1.75,1,1,0
4,3.25,1,2,1


##Make linear model


In [ ]:
Xtr = train_data_transformed.values[:,:-1]
ytr = train_data_transformed["RENDIMIENTO_GLOBAL"].values
print (Xtr.shape, ytr.shape)

(692500, 4) (692500,)


In [ ]:
ytr = ytr.astype(int)

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(Xtr, ytr)

LogisticRegression()

In [ ]:
Xts = test_data_transformed.values[:,:]
print (Xts.shape)

(296786, 4)


In [ ]:
preds_tr = lr.predict(Xtr)
preds_ts = lr.predict(Xts)

In [ ]:
print (preds_tr[:50])
print (preds_ts[:50])

[3 3 3 3 3 3 1 0 2 3 0 0 3 0 1 3 1 0 0 0 0 0 0 0 3 0 0 3 3 0 0 1 3 3 2 3 3
 2 2 3 3 3 3 0 2 3 3 3 3 0]
[0 1 3 0 1 0 3 3 0 0 2 0 3 3 3 1 3 0 1 1 3 0 3 0 2 3 3 0 0 3 1 2 3 0 3 0 3
 0 0 0 0 3 1 0 0 0 0 3 1 3]


In [ ]:
np.mean(preds_tr==ytr)

0.3467581227436823

In [ ]:
# prompt: quiero que crees un archivo para entregar en kaggle, con 2 columnas, el id del test.csv y el resultado que conseguimos en el modelo lineal preds_ts, pero quiero que hagas la inversa de transformar rendimiento, es decir, esos numeros 0 1 2 3 los transformes en bajo, alto, etc

# Reverse the transformation of the performance
reverse_rmap = {3: 'alto', 0: 'bajo', 1: 'medio-bajo', 2: 'medio-alto'}
preds_ts_reversed = [reverse_rmap[pred] for pred in preds_ts]

# Create a DataFrame for submission
submission_df = pd.DataFrame({'ID': w['ID'], 'RENDIMIENTO_GLOBAL': preds_ts_reversed})

# Save the submission file
submission_df.to_csv('submission.csv', index=False)